# PyKX Query Components
Query all components of the application.

## Architecture
<img src="images/Deepdive Diagrams-BasicTick V3.drawio.png"  width="80%">


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from env import *
from managed_kx import *

# Cluster names and database
from config import *


In [2]:
# Using credentials and create service client
session = boto3.Session()

# create finspace client
client = get_client(session=session)

# Connections to Clusters

In [3]:
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)
tp  = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=TP_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
rdb = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
rts = get_pykx_connection(client, 
                           environmentId=ENV_ID, clusterName=RTS_CLUSTER_NAME, 
                           userName=KDB_USERNAME, boto_session=session)
gw = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Query the HDB
With the HDB connection, query its data.

In [4]:
# ensure database is loaded
hdb('.Q.lo[hsym`$.aws.akdbp,"/",.aws.akdb,"/";0b;0b]')

# inventory of tables in the database and rows in each
print("All Tables and Counts")
display( hdb("tables[]!count each value each tables[]") )
print(40*'=')

# Dates and Counts of one table
tables = hdb('tables[]').py()

for t in tables:
    # anything to display?
    tt = hdb(f"select {t}s:count i by date from {t} where date in 10#desc date").pd()
    r = rdb(f'count {t}').py()

    if r == 0: 
        continue

    print(f'{t}: {r:,}')
    print(40*'-')
    display(tt)
    print(40*'=')

All Tables and Counts


,
,
quote,42309983
trade,8461377


quote: 852,991
----------------------------------------


,quotes
date,
2024-10-22,4356637
2024-10-23,4495478
2024-10-24,4401306
2024-10-25,4471510
2024-10-28,4711942
2024-10-29,4619618
2024-10-30,4544274
2024-10-31,4333345
2024-11-01,6375873


trade: 170,724
----------------------------------------


,trades
date,
2024-10-22,872530
2024-10-23,899400
2024-10-24,879672
2024-10-25,894169
2024-10-28,941313
2024-10-29,924403
2024-10-30,907938
2024-10-31,867065
2024-11-01,1274887


# Query the RBD
With the RDB connection, query its data. Will use a q magic cell to send a function to the RDB and run it from Python as well.


In [5]:
# inventory of tables in the database and rows in each
print("Counts")
display( rdb("tables[]!count each value each tables[]") )

# last timestamps in each table
print("Last Times")
display( rdb("tables[]!{exec first max `time$time from x}each tables[]") )
print()
print(40*'=')


# Dates and Counts of one table
tables = rdb('tables[]').py()

for t in tables:
    r = rdb(f'count {t}').py()

    if r == 0: 
        continue

    print(f'{t}: {r:,}')
    print(40*'-')
    # Summarize table by hour
    display( rdb(f"select {t}s:count i by hour:`hh$time from {t}") )
    print(40*'=')

Counts


,
,
quote,852991
trade,170724


Last Times


,
,
quote,18:33:10.948
trade,18:33:10.948



quote: 852,991
----------------------------------------


,quotes
hour,
18i,852991


trade: 170,724
----------------------------------------


,trades
hour,
18i,170724


## Define a function on the RDB
Using a q magic cell, define a function on the RDB.

In [6]:
# get the RDB conneciton string
rdb_conn_str = get_kx_connection_string(client, 
                                  environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME, 
                                   userName=KDB_USERNAME, boto_session=session)

# parse the RDB connection string to its components
host, port, username, password = parse_connection_string(rdb_conn_str)

In [7]:
%%q --host $host --port $port --user $username --pass $password

/ define a function to calculate TWAP
generateTWAP:{[syms;st;et] 
    if[syms~`;syms:exec distinct sym from trade];
    // Calculate statistics from trade and quote tables, join the tables with 
    // appropriate join function in this case a union join 
    quoteMetrics:select avg_spread:avg (ask-bid),twa_spread:(next[time]- time) wavg (ask-bid), avg_size:0.5*avg (asize+bsize),avg_duration:"t"$avg next[time]-time by sym from quote where sym in syms,time within(st;et); 
    tradeMetrics:select std_dev:2*dev price, twap:(next[time]-time) wavg price,max_price:max price, min_price:min price,vwap:size wavg price by sym from trade where sym in syms,time within(st;et); 
    quoteMetrics uj tradeMetrics 
 }


## Call Function on RDB and Display Results
Function is called on the RDB, results are then returned as a Pandas DataFrame using PyKX and display the results as a table in the notebook.

In [8]:
# Call the function for all tickers and a time range
display( rdb("generateTWAP[`;00:00:00.040; 23:59:59.999]") )

# call the function for some tickers and another range
display( rdb("generateTWAP[`AAPL`IBM;00:00:00.040; 23:59:59.999]") )

,avg_spread,twa_spread,avg_size,avg_duration,std_dev,twap,max_price,min_price,vwap
sym,,,,,,,,,
AAPL,1.000919,0.9659765,54.47286,04:18:48.438,2.018066,88.10406,92.08,85.17,87.92174
AIG,1.00018,0.9824703,54.5589,10:46:30.735,0.3967005,27.94617,28.49,27.22,27.89881
AMD,1.00012,1.058223,54.51492,07:12:23.222,0.6451514,32.83806,33.73,31.69,32.84611
DELL,1.000215,1.036736,54.57653,10:47:33.801,0.1806113,12.20689,12.45,11.9,12.19404
DOW,0.9991646,1.030942,54.66211,21:28:52.323,0.2350316,19.96037,20.3,19.68,19.95932
GOOG,1.000407,1.04612,54.52795,03:35:02.110,1.833465,71.41168,74.44,68.62,71.43911
HPQ,1.00052,1.046678,54.38335,10:46:16.430,0.5807981,36.68044,37.47,35.66,36.64234
IBM,1.003691,0.9572093,54.49843,10:48:26.514,0.6536437,41.55742,42.57,40.69,41.56387
INTC,1.000968,1.057939,54.48511,07:14:32.734,1.065274,52.15341,53.75,50.47,52.07772


,avg_spread,twa_spread,avg_size,avg_duration,std_dev,twap,max_price,min_price,vwap
sym,,,,,,,,,
AAPL,1.000919,0.9659765,54.47286,04:18:48.438,2.018066,88.10406,92.08,85.17,87.92174
IBM,1.003691,0.9572093,54.49843,10:48:26.514,0.6536437,41.55742,42.57,40.69,41.56387


# Query the GW

In [9]:
# Query the GW for its connected processes, are all connected?
proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# are any processes not connected? if so-reconnect
if (len(proc_pdf) == 0) or len(proc_pdf[proc_pdf.connected == False].index) > 0:
    print("reinit Gateway")
    gw("reinit[hdb_name; rdb_name]")
    proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# truncate address with elipsis
proc_pdf['address'] = proc_pdf['address'].str.slice(0,110)+"..."

# display table
display(proc_pdf)

,process,handle,connected,address
0,rdb,13,True,:tcps://ip-192-168-4-192.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-4-192.ec2.internal&Port=443&User...
1,hdb,14,True,:tcps://ip-192-168-9-119.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-9-119.ec2.internal&Port=443&User...
2,hdb,15,True,:tcps://ip-192-168-3-224.ec2.internal:443:GATEWAY_basictickdb:Host=ip-192-168-3-224.ec2.internal&Port=443&User...


In [10]:
# query GW using queryData function on gateway

# query and sample specific table for date range (today -3 days to tomorrow)
gw("res: `time xasc queryData[`trade;`;.z.D-3;.z.D+1]").pd()

# first/last 5 rows from res
display( gw("select [3] from res").pd() )
display( gw("select [-3] from res").pd() )

# select for a specific table from ticker with time range (5 days ago to now)
display( hdb(".query.data[`trade;`IBM;.z.P-5D;.z.P]") )

,sym,time,price,size,source
0,SBUX,2024-10-29 09:30:00.000021450,68.86,13,HDB
1,SBUX,2024-10-29 09:30:00.000035619,68.88,35,HDB
2,HPQ,2024-10-29 09:30:00.000052489,39.21,22,HDB


,sym,time,price,size,source
0,SBUX,2024-11-01 18:33:10.948218356,63.89,37,RDB
1,AMD,2024-11-01 18:33:10.948218356,32.73,26,RDB
2,GOOG,2024-11-01 18:33:10.948218356,71.43,86,RDB


,sym,time,price,size,source
,,,,,
0,IBM,2024.10.28D09:30:00.000030592,43.82,81,HDB
1,IBM,2024.10.28D09:30:00.000152320,43.81,82,HDB
2,IBM,2024.10.28D09:30:00.000268184,43.82,15,HDB
3,IBM,2024.10.28D09:30:00.000920274,43.83,21,HDB
4,IBM,2024.10.28D09:30:00.003657754,43.78,36,HDB
5,IBM,2024.10.28D09:30:00.003880730,43.8,59,HDB
6,IBM,2024.10.28D09:30:00.008656896,43.81,79,HDB
7,IBM,2024.10.28D09:30:00.009098791,43.82,66,HDB
8,IBM,2024.10.28D09:30:00.010738124,43.84,50,HDB


# Query the RTS Cluster
The RTS cluster is subscribing to the tickerplant and maintaining another set of tables. Connect to the RTS and show the contents of its tables.

In [11]:
# inventory of tables in the database and rows in each
print("All Tables and Counts")
display( rts("tables[]!count each value each tables[]") )

tables = rts('tables[]').py()

# show contents of tables
for t in tables:
    # anything to display?
    tt = rts(f"select from {t}").pd()
    r = len(tt.index)

    # nothing in table
    if r == 0: 
        continue

    # print table contents
    print(f'{t}: {r:,}')
    print(100*'=')

    # Contents of table, transponse if small
    if r < 20:
        display(tt.T)
    else:
        display(tt)
    print(100*'-')


All Tables and Counts


,
,
quote,0
trade,0
trade_hlcv,15
trade_last,15
trade_vwap,15


trade_hlcv: 15


sym,AAPL,AIG,AMD,DELL,DOW,GOOG,HPQ,IBM,INTC,MSFT,ORCL,PEP,PRU,SBUX,TXN
high,93.06,28.54,34.05,12.53,20.35,75.39,37.68,42.96,54.01,30.67,36.28,22.49,61.18,66.65,18.58
low,83.84,26.85,31.69,11.87,19.50,68.08,35.62,40.42,49.99,28.19,33.98,20.73,57.77,61.32,17.85
close,87.82,27.76,32.73,12.10,19.88,71.43,36.49,41.43,52.79,29.25,35.27,21.64,59.61,63.89,18.28
volume,9409025.00,3735872.00,5614251.00,3762986.00,1882786.00,11261666.00,3757315.00,3771457.00,5633992.00,5590359.00,3753267.00,7502077.00,3721811.00,7506178.00,1892803.00


----------------------------------------------------------------------------------------------------
trade_last: 15


sym,AAPL,AIG,AMD,DELL,DOW,GOOG,HPQ,IBM,INTC,MSFT,ORCL,PEP,PRU,SBUX,TXN
time,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356,2024-11-01 18:33:10.948218356
price,87.82,27.76,32.73,12.1,19.88,71.43,36.49,41.43,52.79,29.25,35.27,21.64,59.61,63.89,18.28
size,88,94,26,28,67,86,85,95,75,50,23,25,92,37,45


----------------------------------------------------------------------------------------------------
trade_vwap: 15


sym,AAPL,AIG,AMD,DELL,DOW,GOOG,HPQ,IBM,INTC,MSFT,ORCL,PEP,PRU,SBUX,TXN
vwap,1.253278e+05,3.975080e+04,4.701007e+04,1.741340e+04,2.854573e+04,1.024583e+05,5.233380e+04,5.951569e+04,7.433518e+04,4.178491e+04,5.050419e+04,3.082326e+04,8.494891e+04,9.150563e+04,2.605667e+04
volume,9.409025e+06,3.735872e+06,5.614251e+06,3.762986e+06,1.882786e+06,1.126167e+07,3.757315e+06,3.771457e+06,5.633992e+06,5.590359e+06,3.753267e+06,7.502077e+06,3.721811e+06,7.506178e+06,1.892803e+06


----------------------------------------------------------------------------------------------------


In [12]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-11-01 18:33:18.008650
